In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 17493, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 17493 (delta 1), reused 0 (delta 0), pack-reused 17490 (from 3)
Receiving objects: 100% (17493/17493), 16.56 MiB | 10.76 MiB/s, done.
Resolving deltas: 100% (11996/11996), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving food_2.v2i.yolov5pytorch.zip to food_2.v2i.yolov5pytorch.zip


In [ ]:
import zipfile
with zipfile.ZipFile("food_2.v2i.yolov5pytorch.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/yolov5/")

In [ ]:
import os
import shutil
import random
from collections import defaultdict
from tqdm import tqdm

# 현재 구조 기준 경로
image_input_dir = "train/images"
label_input_dir = "train/labels"

# 새로 나눌 경로
for split in ["train", "val", "test"]:
    os.makedirs(f"images/{split}", exist_ok=True)
    os.makedirs(f"labels/{split}", exist_ok=True)

# 클래스별 분류
class_to_files = defaultdict(list)

for label_file in os.listdir(label_input_dir):
    label_path = os.path.join(label_input_dir, label_file)
    with open(label_path, "r") as f:
        lines = f.readlines()
        if not lines:
            continue
        class_id = lines[0].strip().split()[0]
        image_file = label_file.replace(".txt", ".jpg")
        if os.path.exists(os.path.join(image_input_dir, image_file)):
            class_to_files[class_id].append(label_file)

# 클래스별 8:1:1 분할
for class_id, label_files in tqdm(class_to_files.items()):
    random.shuffle(label_files)
    total = len(label_files)
    n_train = int(total * 0.8)
    n_val = int(total * 0.1)

    for i, file in enumerate(label_files):
        if i < n_train:
            split = "train"
        elif i < n_train + n_val:
            split = "val"
        else:
            split = "test"

        src_img = os.path.join(image_input_dir, file.replace(".txt", ".jpg"))
        src_lbl = os.path.join(label_input_dir, file)

        dst_img = os.path.join("images", split, file.replace(".txt", ".jpg"))
        dst_lbl = os.path.join("labels", split, file)

        shutil.copy(src_img, dst_img)
        shutil.copy(src_lbl, dst_lbl)

print("✅ 클래스별로 8:1:1 분할 완료")


100%|██████████| 14/14 [00:00<00:00, 54.97it/s]

✅ 클래스별로 8:1:1 분할 완료


In [ ]:
yaml_content = """
train: ./images/train
val: ./images/val
test: ./images/test

nc: 14
names:
  - anchovy
  - bulgogi
  - dotorimuk
  - galbi
  - gosari
  - jangeogui
  - japchae
  - jogaegui
  - kongnamulmuchim
  - macerel
  - potatosalad
  - sausage
  - sigeumchinamul
  - tofu_kimchi
  """

with open("data.yaml", "w", encoding="utf-8") as f:
    f.write(yaml_content)

print("✅ data.yaml 저장 완료!")

✅ data.yaml 저장 완료!


In [ ]:
!python train.py --img 640 --batch 16 --epochs 100 --data data.yaml --weights yolov5s.pt --name food_detect


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  with torch.cuda.amp.autocast(amp):
      62/99      4.56G    0.02777    0.01674   0.004304         55        640:   2% 1/63 [00:00<00:12,  4.77it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      62/99      4.56G    0.02787    0.01908   0.006208         59        640:   3% 2/63 [00:00<00:15,  4.02it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      62/99      4.56G    0.02905    0.01926   0.005641         62        640:   5% 3/63 [00:00<00:16,  3.63it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      62/99      4

In [ ]:
from google.colab import drive
import shutil
import os

# 1. Google Drive 마운트
drive.mount('/content/drive')

# 2. YOLOv5 결과 디렉토리 설정 (현재 실험 이름: food-detect)
exp_name = "food_detect2"
result_dir = f"/content/yolov5/runs/train/{exp_name}/"

# 3. 저장 경로 설정 (Google Drive)
save_dir = "/content/drive/MyDrive/yolov5_models"
os.makedirs(save_dir, exist_ok=True)

# 4. best.pt 저장
shutil.copy(os.path.join(result_dir, "weights", "best.pt"),
            os.path.join(save_dir, f"{exp_name}_best.pt"))

# 5. results.png 저장
shutil.copy(os.path.join(result_dir, "results.png"),
            os.path.join(save_dir, f"{exp_name}_results.png"))

print("✅ best.pt와 results.png 모두 Google Drive에 저장 완료되었습니다.")

Mounted at /content/drive
✅ best.pt와 results.png 모두 Google Drive에 저장 완료되었습니다.
